Практическое задание 4
КМБО-06-20, Бундуки Владислав, Вариант 2

Устанавливаем необходимые библиотеки:

In [ ]:
!pip install pandas
!pip install sklearn

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
data = pd.read_csv('aug_test.csv', index_col='city')

#исключим столбец с уникальными значениями: enrollee_id	
data = data[data.columns.drop(['enrollee_id'])]
data.head()

,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
city,,,,,,,,,,,
city_41,0.827,Male,Has relevent experience,Full time course,Graduate,STEM,9,<10,NaN,1,21
city_103,0.920,Female,Has relevent experience,no_enrollment,Graduate,STEM,5,NaN,Pvt Ltd,1,98
city_21,0.624,Male,No relevent experience,no_enrollment,High School,NaN,<1,NaN,Pvt Ltd,never,15
city_13,0.827,Male,Has relevent experience,no_enrollment,Masters,STEM,11,10/49,Pvt Ltd,1,39
city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,10000+,Pvt Ltd,>4,72


Описание столбцов:
enrollee_id :  уникальный идентификатор кандидата

city: Код города

city_ development _index : индекс развития города (в масштабе)

gender: Пол кандидата

relevent_experience: Соответствующий опыт кандидата

enrolled_university: Тип зачисленных университетских курсов, если таковые имеются.

education_level: Уровень образования кандидата

major_discipline :Обучение основной дисциплине кандидата

experience: Кандидатский общий стаж в годах

company_size: Количество сотрудников в компании текущего работодателя

company_type : Тип текущего работодателя

lastnewjob: разница в годах между предыдущей работой и текущей работой

training_hours: завершенные часы обучения

target: 0 - Не ищу смены работы, 1 - Ищу смену работы

In [2]:
#Выводим информацию о датасете
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2129 entries, city_41 to city_102
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city_development_index  2129 non-null   float64
 1   gender                  1621 non-null   object 
 2   relevent_experience     2129 non-null   object 
 3   enrolled_university     2098 non-null   object 
 4   education_level         2077 non-null   object 
 5   major_discipline        1817 non-null   object 
 6   experience              2124 non-null   object 
 7   company_size            1507 non-null   object 
 8   company_type            1495 non-null   object 
 9   last_new_job            2089 non-null   object 
 10  training_hours          2129 non-null   int64  
dtypes: float64(1), int64(1), object(9)
memory usage: 199.6+ KB


In [4]:
#Кодируем столбец из категориального признака
data['gender'] = np.where(data['gender'] == 'Male', 1, 0)
data['education_level'] = np.where(data['education_level'] == 'Masters', 0, 1)
data['relevent_experience'] = np.where(data['relevent_experience'] == 'No relevent experience',0,1)

In [7]:
my_set = set(data.enrolled_university)
i = 0
for item in my_set:
    data['enrolled_university'] = data['enrolled_university'].replace(item, i)
    i = i + 1

    
my_set = set(data.last_new_job)
i = 0
for item in my_set:
    data['last_new_job'] = data['last_new_job'].replace(item, i)
    i = i + 1
    
my_set = set(data.major_discipline)
i = 0
for item in my_set:
    data['major_discipline'] = data['major_discipline'].replace(item, i)
    i = i + 1
    
my_set = set(data.company_type)
i = 0
for item in my_set:
    data['company_type'] = data['company_type'].replace(item, i)
    i = i + 1
    
my_set = set(data.company_size)
i = 0
for item in my_set:
    data['company_size'] = data['company_size'].replace(item, i)
    i = i + 1
    
my_set = set(data.experience)
i = 0
for item in my_set:
    data['experience'] = data['experience'].replace(item, i)
    i = i + 1

data.loc[data.experience == 'NaN' , 'experience'] = 0

data.loc[data.last_new_job == 'never' , 'last_new_job'] = 0
data.loc[data.last_new_job == 'NaN' , 'last_new_job'] = 0
data.loc[data.last_new_job == '>4' , 'last_new_job'] = 5

data.loc[data.gender == 'NaN' , 'gender'] = 0

,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
city,,,,,,,,,,,
city_41,0.827,1,1,2,1,5,9,3,0,6,21
city_103,0.920,0,1,3,1,5,5,0,3,6,98
city_21,0.624,1,0,3,1,0,17,0,3,4,15
city_13,0.827,1,1,3,0,5,7,1,3,6,39
city_103,0.920,1,1,3,1,5,15,8,3,5,72


Выделяем целевую переменную

In [9]:
target = data.education_level
train = data.drop('education_level', axis = 1) 

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2129 entries, city_41 to city_102
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city_development_index  2129 non-null   float64
 1   gender                  2129 non-null   int32  
 2   relevent_experience     2129 non-null   int32  
 3   enrolled_university     2129 non-null   int64  
 4   major_discipline        2129 non-null   int64  
 5   experience              2129 non-null   int64  
 6   company_size            2129 non-null   int64  
 7   company_type            2129 non-null   int64  
 8   last_new_job            2129 non-null   int64  
 9   training_hours          2129 non-null   int64  
dtypes: float64(1), int32(2), int64(7)
memory usage: 166.3+ KB


Выделяем тренировочную и тестовую выборки

In [11]:
#y - целевая переменнная (target)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size = 0.3, random_state = 42)

N_train, _ = X_train.shape 
N_test,  _ = X_test.shape 
print (N_train, N_test)

1490 639


Построим классификатор логистической регрессии

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

grid={"C":np.logspace(-5,5,11), "penalty":["l1","l2"]}
logreg=LogisticRegression(solver='liblinear')
logreg_cv=GridSearchCV(logreg,grid,cv=10)
logreg_cv.fit(X_train,y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(solver='liblinear'),
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05]),
                         'penalty': ['l1', 'l2']})

In [13]:
logreg_cv.best_estimator_ 

LogisticRegression(C=1e-05, solver='liblinear')

Оценим точность построенного классификатора с помощью метрик precision, recall и F1

In [15]:
from sklearn.model_selection import cross_val_score
print("f1:"+str(np.average(cross_val_score(logreg_cv, X_train, y_train, scoring='f1'))))
print("precision:"+str(np.average(cross_val_score(logreg_cv, X_train, y_train, scoring='precision'))))
print("recall:"+str(np.average(cross_val_score(logreg_cv, X_train, y_train, scoring='recall'))))

f1:0.872065647018388
precision:0.7731543624161075
recall:1.0


In [16]:
from sklearn.model_selection import cross_val_score
print("f1:"+str(np.average(cross_val_score(logreg_cv, X_test, y_test, scoring='f1'))))
print("precision:"+str(np.average(cross_val_score(logreg_cv, X_test, y_test, scoring='precision'))))
print("recall:"+str(np.average(cross_val_score(logreg_cv, X_test, y_test, scoring='recall'))))

f1:0.8589274681471991
precision:0.7527436023622047
recall:1.0


Построим классификатор типа Случайный Лес (Random Forest) для решения той же задачи классификации

In [17]:
from sklearn.ensemble import RandomForestClassifier 
param_grid = { 'n_estimators': [50, 100, 150],'max_features': ['auto'],'max_depth' : list(range(1, 10)), 'criterion' :['gini']}
RandForCrit = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid, cv = 5, refit = True) 
RandForCrit.fit(X_train, y_train)
RandForCrit.predict(X_test)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

Оценим точность построенного классификатора с помощью метрик precision, recall и F1

In [18]:
print("f1:"+str(np.average(cross_val_score(RandForCrit.best_estimator_, X_train, y_train, scoring='f1'))))
print("precision:"+str(np.average(cross_val_score(RandForCrit.best_estimator_, X_train, y_train, scoring='precision'))))
print("recall:"+str(np.average(cross_val_score(RandForCrit.best_estimator_, X_train, y_train, scoring='recall'))))

f1:0.878701665883123
precision:0.7870505745429996
recall:0.9904611330698287


Оценим точность построенного классификатора с помощью метрик precision, recall и F1 на тестовой выборке

Построим классификатор типа Случайный Лес (Random Forest) для решения той же задачи классификации

Оценим точность построенного классификатора с помощью метрик precision, recall и F1 на тестовой выборке

In [19]:
print("f1:"+str(np.average(cross_val_score(RandForCrit.best_estimator_, X_test, y_test, scoring='f1'))))
print("precision:"+str(np.average(cross_val_score(RandForCrit.best_estimator_, X_test, y_test, scoring='precision'))))
print("recall:"+str(np.average(cross_val_score(RandForCrit.best_estimator_, X_test, y_test, scoring='recall'))))

f1:0.8525014473313075
precision:0.7535815669382789
recall:0.9813359106529209


Сравним результаты:

f1 - лучше у классификатора Логистической регрессии

precision - лучше у классификатора Случайного леса

recall - лучше у классификатора Логистической регрессии